# Введение в RL и пакет Gymnasium

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://gymnasium.farama.org/
* https://pypi.org/project/ufal.pybox2d/
* https://gymnasium.farama.org/tutorials/gymnasium_basics/environment_creation/
* https://gymnasium.farama.org/api/spaces/fundamental/
* https://gymnasium.farama.org/environments/toy_text/blackjack/

## Задачи для совместного разбора

1\. Рассмотрите пример создания окружения `gymnasium` и основные этапы взаимодействия с этим окружением.

<img src="https://gymnasium.farama.org/_images/AE_loop.png" width="300"/>

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Создайте окружение `Blackjack-v1`. Сыграйте `N=10000` игр, выбирая действие случайным образом. Посчитайте и выведите на экран долю выигранных игр.

- [ ] Проверено на семинаре

In [ ]:
import gymnasium as gym
from tqdm import tqdm

env = gym.make('Blackjack-v1', render_mode=None)

N_GAMES = 10_000
wins = 0
draws = 0
losses = 0

for _ in range(N_GAMES):
    state, info = env.reset()

    terminated = False
    truncated = False

    while not (terminated or truncated):
        action = env.action_space.sample()

        next_state, reward, terminated, truncated, info = env.step(action)

        if terminated or truncated:
            if reward == 1.0:
                wins += 1
            elif reward == 0.0:
                draws += 1
            else: # reward == -1.0
                losses += 1

env.close()
win_rate = wins / N_GAMES
print(f"Сыграно игр: {N_GAMES}")
print(f"Побед: {wins}")
print(f"Ничьих: {draws}")
print(f"Поражений: {losses}")
print(f"Доля выигрышей (Win Rate): {win_rate:.4f} ({win_rate * 100:.2f}%)")

Сыграно игр: 10000
Побед: 2803
Ничьих: 439
Поражений: 6758
Доля выигрышей (Win Rate): 0.2803 (28.03%)


<p class="task" id="2"></p>

2\. Создайте окружение `Blackjack-v1`. Предложите стратегию, которая позволит, в среднем, выигрывать чаще, чем случайный выбор действия. Реализуйте эту стратегию и сыграйте `N=10000` игр, выбирая действие согласно этой стратегии. Посчитайте и выведите на экран долю выигранных игр.

- [ ] Проверено на семинаре

In [ ]:
def simple_strategy(observation):

    player_sum, dealer_card, usable_ace = observation
    if player_sum < 17:
        return 1
    else:
        return 0

env = gym.make('Blackjack-v1', render_mode=None)
N_GAMES = 10_000
wins = 0
draws = 0
losses = 0

for _ in range(N_GAMES):
    state, info = env.reset()
    terminated = False
    truncated = False

    while not (terminated or truncated):
        action = simple_strategy(state)

        state, reward, terminated, truncated, info = env.step(action)

        if terminated or truncated:
            if reward == 1.0:
                wins += 1
            elif reward == 0.0:
                draws += 1
            else:
                losses += 1

env.close()

win_rate = wins / N_GAMES
print(f"Сыграно игр: {N_GAMES}")
print(f"Побед: {wins}")
print(f"Ничьих: {draws}")
print(f"Доля выигрышей (Win Rate): {win_rate:.4f} ({win_rate * 100:.2f}%)")


Сыграно игр: 10000
Побед: 4119
Ничьих: 1009
Доля выигрышей (Win Rate): 0.4119 (41.19%)


<p class="task" id="3"></p>

3\. Создайте окружение для игры в крестики-нолики, реализовав интерфейс `gym.Env`. Решение должно удовлетворять следующим условиям:
* для создания пространства состояний используется `spaces.Box`;
* для создания пространства действий используется `spaces.MultiDiscrete`;
* игра прекращается, если:
    - нет возможности сделать ход;
    - игрок пытается отметить уже выбранную ячейку.
* после каждого хода игрок получает награду:
    - 0, если игра не закончена;
    - 1, если игрок выиграл;
    - -1, если игрок проиграл.
* стратегию выбора действия для второго игрока (машины) определите самостоятельно.

Стратегия поведения машины является частью окружения и должна быть реализована внутри него. Сделайте все соответствующие переменные и методы приватными (названия всех переменных начинаются с `__`), подчеркнув, что у пользователя не должно быть к ним доступа извне.

Сыграйте одну игру, выбирая действия случайным образом. Выведите на экран состояние окружения после каждого хода и итоговую награду пользователя за сессию.

- [ ] Проверено на семинаре

In [ ]:
from gymnasium import spaces
import numpy as np
import random

class TicTacToeEnv(gym.Env):
    def __init__(self):
        super().__init__()

        self.action_space = spaces.MultiDiscrete([3, 3])
        self.observation_space = spaces.Box(
            low=0, high=2, shape=(3, 3), dtype=np.int8
        )

        self.__board = None
        self.__player_mark = 1
        self.__bot_mark = 2

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        self.__board = np.zeros((3, 3), dtype=np.int8)

        return self.__board, {}

    def step(self, action):
        row, col = action

        if self.__board[row, col] != 0:
            return self.__board, -1, True, False, {"info": "Invalid move"}

        self.__board[row, col] = self.__player_mark

        if self.__check_winner(self.__player_mark):
            return self.__board, 1, True, False, {"info": "Player won"}

        if self.__is_full():
            return self.__board, 0, True, False, {"info": "Draw"}

        self.__bot_step()

        if self.__check_winner(self.__bot_mark):
            return self.__board, -1, True, False, {"info": "Bot won"}

        if self.__is_full():
            return self.__board, 0, True, False, {"info": "Draw"}

        return self.__board, 0, False, False, {}

    def render(self):
        print("-" * 5)
        for row in self.__board:
            print('|'.join(['X' if x==1 else 'O' if x==2 else ' ' for x in row]))
        print("-" * 5)


    def __bot_step(self):
        empty_cells = np.argwhere(self.__board == 0)
        if len(empty_cells) > 0:
            chosen_idx = self.np_random.integers(0, len(empty_cells))
            move = empty_cells[chosen_idx]
            self.__board[move[0], move[1]] = self.__bot_mark

    def __check_winner(self, mark):
        for i in range(3):
            if np.all(self.__board[i, :] == mark) or np.all(self.__board[:, i] == mark):
                return True
        if self.__board[0, 0] == mark and self.__board[1, 1] == mark and self.__board[2, 2] == mark:
            return True
        if self.__board[0, 2] == mark and self.__board[1, 1] == mark and self.__board[2, 0] == mark:
            return True
        return False

    def __is_full(self):
        return not np.any(self.__board == 0)
env = TicTacToeEnv()

print("Начинаем игру в Крестики-Нолики!")
obs, _ = env.reset()
env.render()

done = False
total_reward = 0

while not done:
    action = env.action_space.sample()
    print(f"Игрок выбирает клетку: {action}")

    obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    total_reward += reward

    env.render()
    if done:
        print(f"Игра окончена. Награда: {reward}")
        print(f"Инфо: {info}")

env.close()

Начинаем игру в Крестики-Нолики!
-----
 | | 
 | | 
 | | 
-----
Игрок выбирает клетку: [2 0]
-----
O| | 
 | | 
X| | 
-----
Игрок выбирает клетку: [0 0]
-----
O| | 
 | | 
X| | 
-----
Игра окончена. Награда: -1
Инфо: {'info': 'Invalid move'}


<p class="task" id="4"></p>

4\. Предложите стратегию (в виде алгоритма без использования методов машинного обучения), которая позволит, в среднем, выигрывать в крестики-нолики чаще, чем случайный выбор действия. Реализуйте эту стратегию и сыграйте игру, выбирая действия согласно этой стратегии. Выведите на экран состояние окружения после каждого хода и итоговую награду пользователя за сессию.

- [ ] Проверено на семинаре

In [ ]:

import time

def check_potential_win(board, mark):
    empty_cells = np.argwhere(board == 0)

    for cell in empty_cells:
        r, c = cell
        temp_board = board.copy()
        temp_board[r, c] = mark

        if np.all(temp_board[r, :] == mark) or np.all(temp_board[:, c] == mark):
            return (r, c)

        if (r == c) or (r + c == 2):
            if (temp_board[0, 0] == mark and temp_board[1, 1] == mark and temp_board[2, 2] == mark):
                return (r, c)
            if (temp_board[0, 2] == mark and temp_board[1, 1] == mark and temp_board[2, 0] == mark):
                return (r, c)

    return None

def smart_agent_action(board):
    PLAYER_MARK = 1
    BOT_MARK = 2
    win_move = check_potential_win(board, PLAYER_MARK)
    if win_move is not None:
        return np.array(win_move)

    block_move = check_potential_win(board, BOT_MARK)
    if block_move is not None:
        return np.array(block_move)
    if board[1, 1] == 0:
        return np.array([1, 1])

    corners = [[0, 0], [0, 2], [2, 0], [2, 2]]
    available_corners = [c for c in corners if board[c[0], c[1]] == 0]
    if len(available_corners) > 0:
        idx = np.random.randint(len(available_corners))
        return np.array(available_corners[idx])

    empty_cells = np.argwhere(board == 0)
    if len(empty_cells) > 0:
        idx = np.random.randint(len(empty_cells))
        return empty_cells[idx]

    return np.array([0, 0])

env = TicTacToeEnv()

print("=== Начинаем игру: Умный Агент vs Рандомный Бот ===")
obs, _ = env.reset()
env.render()

done = False

while not done:
    action = smart_agent_action(obs)
    print(f"\nАгент ходит в: {action}")

    obs, reward, terminated, truncated, info = env.step(action)
    env.render()

    done = terminated or truncated

    if done:
        if reward == 1:
            print("\nИтог: ПОБЕДА Агента!")
        elif reward == -1:
            print("\nИтог: ПОРАЖЕНИЕ (Бот выиграл)!")
        else:
            print("\nИтог: НИЧЬЯ!")

env.close()

=== Начинаем игру: Умный Агент vs Рандомный Бот ===
-----
 | | 
 | | 
 | | 
-----

Агент ходит в: [1 1]
-----
 | | 
O|X| 
 | | 
-----

Агент ходит в: [0 0]
-----
X| | 
O|X| 
O| | 
-----

Агент ходит в: [2 2]
-----
X| | 
O|X| 
O| |X
-----

Итог: ПОБЕДА Агента!


<p class="task" id="5"></p>

5\. Создайте окружение `MountainCar-v0`. Проиграйте 10 эпизодов и сохраните на диск файл с записью каждого пятого эпизода. Для записи видео воспользуйтесь обёрткой `RecordVideo`. Вставьте скриншот, на котором видно, что файлы были созданы.

- [ ] Проверено на семинаре

In [ ]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
import os

base_env = gym.make('MountainCar-v0', render_mode='rgb_array')

trigger = lambda episode_id: episode_id % 5 == 0
env = RecordVideo(base_env, video_folder="./video_mountain_car", episode_trigger=trigger, disable_logger=True)

N_EPISODES = 50

print("Начинаем симуляцию...")

for i in range(N_EPISODES):
    obs, info = env.reset()
    terminated = truncated = False

    while not (terminated or truncated):
        action = env.action_space.sample()

        obs, reward, terminated, truncated, info = env.step(action)
    if trigger(i):
        print(f"Эпизод {i} завершен. Видео должно быть записано.")

env.close()
try:
    files = os.listdir("./video_mountain_car")
    mp4_files = [f for f in files if f.endswith('.mp4')]
    print(f"Найдены видеофайлы: {mp4_files}")
except FileNotFoundError:
    print("Папка с видео не найдена. Что-то пошло не так.")

D:\Conda_Envs\aio_project_d\lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at D:\Jupik\video_mountain_car folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Начинаем симуляцию...
Эпизод 0 завершен. Видео должно быть записано.
Эпизод 5 завершен. Видео должно быть записано.
Эпизод 10 завершен. Видео должно быть записано.
Эпизод 15 завершен. Видео должно быть записано.
Эпизод 20 завершен. Видео должно быть записано.
Эпизод 25 завершен. Видео должно быть записано.
Эпизод 30 завершен. Видео должно быть записано.
Эпизод 35 завершен. Видео должно быть записано.
Эпизод 40 завершен. Видео должно быть записано.
Эпизод 45 завершен. Видео должно быть записано.
Найдены видеофайлы: ['rl-video-episode-0.mp4', 'rl-video-episode-10.mp4', 'rl-video-episode-15.mp4', 'rl-video-episode-20.mp4', 'rl-video-episode-25.mp4', 'rl-video-episode-30.mp4', 'rl-video-episode-35.mp4', 'rl-video-episode-40.mp4', 'rl-video-episode-45.mp4', 'rl-video-episode-5.mp4']
